In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import re

In [2]:
def status_check(url):
    import requests
    r=requests.get(url)
    if r.status_code<300:
        print("request was successful")
    elif r.status_code>=400 and r.status_code<500:
        print('request failed because it is non-existant or forbidden')
    else: print('blablabla')
    return r

In [3]:
def get_soup(url):
    r=status_check(url)
    soup = BeautifulSoup(r.content)
    return soup

In [4]:
def get_stars(soup):
    if len(soup.select('h4:contains("Stars")~a'))>1:
        stars= [i.text for i in soup.select('h4:contains("Stars")~a')[:-1]]
    else:
        stars = np.nan
    return stars

In [5]:
def get_writer(soup):
    if len(soup.select('h4:contains("Writer")+a'))>0:
        writer = soup.select('h4:contains("Writer")+a')[0].text
    else:
        writer = np.nan
    return writer

In [6]:
def get_popularity(soup):
    if len(soup.select('div:contains("Popularity")+div>span'))>0:
        popularity = soup.select('div:contains("Popularity")+div>span')[0].text.strip()
    else:
        popularity = np.nan
    return popularity

In [7]:
def get_keywords(soup):
    if len(soup.select('h4:contains("Keywords")~a'))>0:
        plot_keywords = [i.text for i in soup.select('h4:contains("Keywords")~a')]
    else:
        plot_keywords = np.nan
    return plot_keywords

In [8]:
def get_languages(soup):
    if len(soup.select('h4:contains("Language")~a'))>0:
        languages = [i.text for i in soup.select('h4:contains("Language")~a')]
    else:
        languages = np.nan
    return languages

In [9]:
def get_country(soup):
    if len(soup.select('h4:contains("Country")+a'))>0:
        country = soup.select('h4:contains("Country")+a')[0].text
    else:
        country = np.nan
    return country

In [10]:
def get_budget(soup):
    if len(soup.select('div.txt-block:contains("Budget")'))>0:
        test=soup.select('div.txt-block:contains("Budget")')[0].text
        budget=re.findall('Budget:\$([\d\,]*)',test)
    else:
        budget = np.nan
    return budget

In [11]:
def get_cum_gross(soup):
    if len(soup.select('div.txt-block:contains("Worldwide")'))>0:
        test2=soup.select('div.txt-block:contains("Worldwide")')[0].text
        gross=re.findall('Gross: \$([\d\,]*)',test2)
    else:
        gross = np.nan
    return gross

In [12]:
df1=pd.read_csv('movies_imdb.csv')

In [13]:
df1.head()

,movie,year,imdbID,genre,runtime,rating,metascore,director,votes,gross
0,The Dark Knight: Le chevalier noir,(2008),/title/tt0468569/,"\nAction, Crime, Drama",152 min,9.0,84.0,Christopher Nolan,"2,161,024",$534.86M
1,Inception,(2010),/title/tt1375666/,"\nAction, Adventure, Sci-Fi",148 min,8.8,74.0,Christopher Nolan,"1,909,234",$292.58M
2,Le seigneur des anneaux: La communauté de l'an...,(2001),/title/tt0120737/,"\nAdventure, Drama, Fantasy",178 min,8.8,92.0,Peter Jackson,"1,561,138",$315.54M
3,Le seigneur des anneaux: Le retour du roi,(2003),/title/tt0167260/,"\nAdventure, Drama, Fantasy",201 min,8.9,94.0,Peter Jackson,"1,549,252",$377.85M
4,The Dark Knight Rises,(2012),/title/tt1345836/,"\nAction, Thriller",164 min,8.4,78.0,Christopher Nolan,"1,431,808",$448.14M


In [14]:
df1.shape

(6735, 10)

In [15]:
ID_list = df1.imdbID.to_list()

In [16]:
ll=[]
n=1

In [111]:
for ID in ID_list:
    row = [ID]
    url='https://www.imdb.com'+ID
    print(n)
    soup = get_soup(url)
    row.append(get_stars(soup))
    row.append(get_writer(soup))
    row.append(get_popularity(soup))
    row.append(get_keywords(soup))
    row.append(get_languages(soup))
    row.append(get_country(soup))
    row.append(get_budget(soup))
    row.append(get_cum_gross(soup))
    ll.append(row)
    n+=1
    time.sleep(0.5)

1696
request was successful
1697
request was successful
1698
request was successful
1699
request was successful
1700
request was successful
1701
request was successful
1702
request was successful
1703
request was successful
1704
request was successful
1705
request was successful
1706
request was successful
1707
request was successful
1708
request was successful
1709
request was successful
1710
request was successful
1711
request was successful
1712
request was successful
1713
request was successful
1714
request was successful
1715
request was successful
1716
request was successful
1717
request was successful
1718
request was successful
1719
request was successful
1720
request was successful
1721
request was successful
1722
request was successful
1723
request was successful
1724
request was successful
1725
request was successful
1726
request was successful
1727
request was successful
1728
request was successful
1729
request was successful
1730
request was successful
1731
request was suc

1989
request was successful
1990
request was successful
1991
request was successful
1992
request was successful
1993
request was successful
1994
request was successful
1995
request was successful
1996
request was successful
1997
request was successful
1998
request was successful
1999
request was successful
2000
request was successful
2001
request was successful
2002
request was successful
2003
request was successful
2004
request was successful
2005
request was successful
2006
request was successful
2007
request was successful
2008
request was successful
2009
request was successful
2010
request was successful
2011
request was successful
2012
request was successful
2013
request was successful
2014
request was successful
2015
request was successful
2016
request was successful
2017
request was successful
2018
request was successful
2019
request was successful
2020
request was successful
2021
request was successful
2022
request was successful
2023
request was successful
2024
request was suc

2282
request was successful
2283
request was successful
2284
request was successful
2285
request was successful
2286
request was successful
2287
request was successful
2288
request was successful
2289
request was successful
2290
request was successful
2291
request was successful
2292
request was successful
2293
request was successful
2294
request was successful
2295
request was successful
2296
request was successful
2297
request was successful
2298
request was successful
2299
request was successful
2300
request was successful
2301
request was successful
2302
request was successful
2303
request was successful
2304
request was successful
2305
request was successful
2306
request was successful
2307
request was successful
2308
request was successful
2309
request was successful
2310
request was successful
2311
request was successful
2312
request was successful
2313
request was successful
2314
request was successful
2315
request was successful
2316
request was successful
2317
request was suc

2575
request was successful
2576
request was successful
2577
request was successful
2578
request was successful
2579
request was successful
2580
request was successful
2581
request was successful
2582
request was successful
2583
request was successful
2584
request was successful
2585
request was successful
2586
request was successful
2587
request was successful
2588
request was successful
2589
request was successful
2590
request was successful
2591
request was successful
2592
request was successful
2593
request was successful
2594
request was successful
2595
request was successful
2596
request was successful
2597
request was successful
2598
request was successful
2599
request was successful
2600
request was successful
2601
request was successful
2602
request was successful
2603
request was successful
2604
request was successful
2605
request was successful
2606
request was successful
2607
request was successful
2608
request was successful
2609
request was successful
2610
request was suc

2868
request was successful
2869
request was successful
2870
request was successful
2871
request was successful
2872
request was successful
2873
request was successful
2874
request was successful
2875
request was successful
2876
request was successful
2877
request was successful
2878
request was successful
2879
request was successful
2880
request was successful
2881
request was successful
2882
request was successful
2883
request was successful
2884
request was successful
2885
request was successful
2886
request was successful
2887
request was successful
2888
request was successful
2889
request was successful
2890
request was successful
2891
request was successful
2892
request was successful
2893
request was successful
2894
request was successful
2895
request was successful
2896
request was successful
2897
request was successful
2898
request was successful
2899
request was successful
2900
request was successful
2901
request was successful
2902
request was successful
2903
request was suc

...

4040
request was successful
4041
request was successful
4042
request was successful
4043
request was successful
4044
request was successful
4045
request was successful
4046
request was successful
4047
request was successful
4048
request was successful
4049
request was successful
4050
request was successful
4051
request was successful
4052
request was successful
4053
request was successful
4054
request was successful
4055
request was successful
4056
request was successful
4057
request was successful
4058
request was successful
4059
request was successful
4060
request was successful
4061
request was successful
4062
request was successful
4063
request was successful
4064
request was successful
4065
request was successful
4066
request was successful
4067
request was successful
4068
request was successful
4069
request was successful
4070
request was successful
4071
request was successful
4072
request was successful
4073
request was successful
4074
request was successful
4075
request was suc

4333
request was successful
4334
request was successful
4335
request was successful
4336
request was successful
4337
request was successful
4338
request was successful
4339
request was successful
4340
request was successful
4341
request was successful
4342
request was successful
4343
request was successful
4344
request was successful
4345
request was successful
4346
request was successful
4347
request was successful
4348
request was successful
4349
request was successful
4350
request was successful
4351
request was successful
4352
request was successful
4353
request was successful
4354
request was successful
4355
request was successful
4356
request was successful
4357
request was successful
4358
request was successful
4359
request was successful
4360
request was successful
4361
request was successful
4362
request was successful
4363
request was successful
4364
request was successful
4365
request was successful
4366
request was successful
4367
request was successful
4368
request was suc

4626
request was successful
4627
request was successful
4628
request was successful
4629
request was successful
4630
request was successful
4631
request was successful
4632
request was successful
4633
request was successful
4634
request was successful
4635
request was successful
4636
request was successful
4637
request was successful
4638
request was successful
4639
request was successful
4640
request was successful
4641
request was successful
4642
request was successful
4643
request was successful
4644
request was successful
4645
request was successful
4646
request was successful
4647
request was successful
4648
request was successful
4649
request was successful
4650
request was successful
4651
request was successful
4652
request was successful
4653
request was successful
4654
request was successful
4655
request was successful
4656
request was successful
4657
request was successful
4658
request was successful
4659
request was successful
4660
request was successful
4661
request was suc

4919
request was successful
4920
request was successful
4921
request was successful
4922
request was successful
4923
request was successful
4924
request was successful
4925
request was successful
4926
request was successful
4927
request was successful
4928
request was successful
4929
request was successful
4930
request was successful
4931
request was successful
4932
request was successful
4933
request was successful
4934
request was successful
4935
request was successful
4936
request was successful
4937
request was successful
4938
request was successful
4939
request was successful
4940
request was successful
4941
request was successful
4942
request was successful
4943
request was successful
4944
request was successful
4945
request was successful
4946
request was successful
4947
request was successful
4948
request was successful
4949
request was successful
4950
request was successful
4951
request was successful
4952
request was successful
4953
request was successful
4954
request was suc

KeyboardInterrupt: 

In [140]:
df2=pd.DataFrame(ll)

In [141]:
df2.columns=['imdbID','stars','writer','popularity','plot_keywords','languages','country','budget','cum_worldwide_gross']

In [142]:
df2.to_csv('movies_imdb_more_info.csv', index=False)

In [146]:
df2.head()

,imdbID,stars,writer,popularity,plot_keywords,languages,country,budget,cum_worldwide_gross
0,/title/tt0468569/,"[Christian Bale, Heath Ledger, Aaron Eckhart]",Jonathan Nolan,125\n ( 36),"[ dc comics, joker, psychopath, clown, cri...","[English, Mandarin]",USA,"[185,000,000]","[1,004,934,033]"
1,/title/tt1375666/,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elle...",Christopher Nolan,89\n ( 68),"[ dream, subconscious, ambiguous ending, th...","[English, Japanese, French]",USA,"[160,000,000]","[829,895,144]"
2,/title/tt0120737/,"[Elijah Wood, Ian McKellen, Orlando Bloom]",J.R.R. Tolkien,113\n ( 14),"[ ring, quest, hobbit, elf, orc]","[English, Sindarin]",New Zealand,"[93,000,000]","[872,491,916]"
3,/title/tt0167260/,"[Elijah Wood, Viggo Mortensen, Ian McKellen]",J.R.R. Tolkien,173\n ( 63),"[ orc, epic, battle, journey, hobbit]","[English, Quenya, Old English, Sindarin]",New Zealand,"[94,000,000]","[1,120,424,614]"
4,/title/tt1345836/,"[Christian Bale, Tom Hardy, Anne Hathaway]",Jonathan Nolan,175\n ( 171),"[ dc comics, batman character, terrorist plo...","[English, Arabic]",UK,"[250,000,000]","[1,081,041,287]"
